In [1]:
import os, sqlite3, xlrd
import pandas as pd

In [13]:
xls_file = pd.ExcelFile('query.xlsx')
table = xls_file.parse('All Samples')
chrom = [x for x in table['Chr'].unique()[:-1]]
table.columns

Index(['Sample', 'Gene', 'Variant', 'Chr', 'Coordinate', 'Variant Length',
       'Type', 'Genotype', 'Exonic', 'Filters', 'Quality', 'GQX',
       'Alternate Alleles', 'Inherited From', 'Alt Variant Freq', 'Read Depth',
       'Alt Read Depth', 'Allelic Depths', 'Custom Annotation',
       'Custom Annotation 2', 'Custom Annotation 3', 'Custom Annotation 4',
       'Custom Gene Annotation', 'Custom Gene Annotation 2', 'Classification',
       'Google Scholar', 'PubMed', 'UCSC Browser', 'Num Transcripts',
       'Transcript', 'Consequence', 'cDNA Position', 'CDS Position',
       'Protein Position', 'Amino Acids', 'Codons', 'Exon', 'Intron',
       'Transcript HGNC', 'Distance', 'Canonical', 'Sift', 'PolyPhen', 'ENSP',
       'HGVSc', 'HGVSp', 'dbSNP ID', 'Ancestral Allele', 'Allele Freq',
       'Allele Freq Global Minor', 'Global Minor Allele', 'Allele Freq Amr',
       'Allele Freq Asn', 'Allele Freq Af', 'Allele Freq Eur',
       'Allele Freq Evs', 'EVS Coverage', 'EVS Samples', 'Co

In [ ]:
#df = table.set_index(['Chr'], drop = False)
# table.index
for i in table.index:
    for chromosome in chrom:
        if table.iloc[i][chromosome] == 1:
            coordina = table.iloc[i]['Coordinate']
            conn = sqlite3.connect('dbase_Sqlite')
            df = pd.read_sql_query("SELECT * FROM Indian WHERE POS=" + str(coordina) , conn)
            print(df)